<a href="https://colab.research.google.com/github/santiagosanchez15/Project1-comp215/blob/main/Comp_215_prject_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1 comp 215

**Author:** Santiago Sanchez Covarrubias

**resources**: *Think python, Claude.ai*
  - https://allendowney.github.io/ThinkPython/. Think python URL


**Objectives**
- The creation of a SLM capable of to predict the third word

**Project description**

The project will develop a SLM capable of predicting the third word.

This project will be focus not only on developing the SLM but also on documenting the process.
Starting by adding different sections, that at the end of different sections will join all the pieces together.

After the SLM has been built with feeded data, the final SLM will be created by inhereting everything from the first one, the difference is tyhat this final version will not only take the feeded data through files but aslo through the Wikimedia REST API, the perfect source for thousand of wirtten texts.

At the end of all the documentation the full code will be available.





# Parsed and clean function

  - get_text:
    - will get the text from the file

  - clean_text
    - will iterate through all the text, check if there are punctuations signs, remove and create a new list of words

  - bigrams -> words
    - check the anount aof times a word repeats after an specific word, then added to dictionary
      "Hello how are you"
      1 - (Hello how)
      2- (how are)
      3- (are you)
      amount of times the second word will come after the first one
      {"Hello", {"how": 2, "this": 4}} etc.


  - trigrams -> 3 words
    - same like bigrams but instead teh combination of 3 words
      next two words plus the word checking such as
      "The castle is big and made of stone"
      1 - (the castle is)
      2- (castle is big)
      3- (is big and)
      aount of times the next two words will come after the first one
      {'The': {"castle is", 3}
      {"The': {"red carpet", 2}
      etc, next two words following the first one there fore key can be tuple